### Seleção, Cruzamento e Mutação

In [19]:
import random
import numpy as np
import pandas as pd

def gerar_cromossomo():
    cromossomo = []
    
    # Gera posições iniciais (0-29)
    posicoes_iniciais = list(range(30))
    random.shuffle(posicoes_iniciais)
    cromossomo.extend(posicoes_iniciais)
    
    # Gera posições finais (0-29)
    posicoes_finais = list(range(30))
    random.shuffle(posicoes_finais)
    cromossomo.extend(posicoes_finais)
    
    # Gera ordem de movimentação (0-29)
    ordem_movimentacao = list(range(30))
    random.shuffle(ordem_movimentacao)
    cromossomo.extend(ordem_movimentacao)
    
    return cromossomo

def cromossomo_para_dataframe(cromossomo):
    container = list(range(30))
    posicao_inicial = cromossomo[:30]
    posicao_final = cromossomo[30:60]
    ordem_movimentacao = cromossomo[60:90]
    
    df = pd.DataFrame({
        'Container': container,
        'Posição Inicial': posicao_inicial,
        'Posição Final': posicao_final,
        'Ordem de Movimentação': ordem_movimentacao
    })
    
    # Ordenar o DataFrame pela coluna 'Ordem de Movimentação'
    df = df.sort_values(by='Ordem de Movimentação')
    
    return df

def inicializar_populacao(tamanho_populacao):
    populacao = []
    for _ in range(tamanho_populacao):
        cromossomo = gerar_cromossomo()
        populacao.append(cromossomo)
    return populacao

def verifica_repetidos(cromossomo):
    intervalos = [
        (0, 29),
        (30, 59),
        (60, 89)
    ]
    
    for intervalo in intervalos:
        valores = cromossomo[intervalo[0]:intervalo[1]+1]
        if len(valores) != len(set(valores)):
            return True
    
    return False

def preencher_porto(df_cromossomo):
    # Inicializar o porto e balsa
    porto = np.zeros((2, 5, 3), dtype=int)
    balsa = np.zeros((3, 4, 3), dtype=int)
    porto_preenchido = np.ones((2, 5, 3), dtype=int)
    cal_movimentos = 0
    
    # Preencher o porto com os dados do cromossomo
    ordem_inicial = []
    ordem_final = []

    df_cromossomo = df_cromossomo.sort_values(by='Posição Inicial')
    for _, linha in df_cromossomo.iterrows():
        ordem_inicial.append(linha['Container'])
    idx = 0
    for andar in range(2):
        for x in range(5):
            for y in range(3):
                if idx < 30:
                    porto[andar, x, y] = ordem_inicial[idx]
                    idx += 1
    
    df_cromossomo = df_cromossomo.sort_values(by='Ordem de Movimentação')
    # Realizar movimentações
    for _, linha in df_cromossomo.iterrows():
        container = linha['Container']
        a_pos_inicial = linha['Posição Inicial']
        a_pos_final = linha['Posição Final']
        pos_inicial = linha['Posição Inicial'] - 1
        pos_final = linha['Posição Final'] - 1
        
        andar_ini, x_ini, y_ini = pos_inicial // 15, (pos_inicial % 15) // 3, (pos_inicial % 15) % 3
        andar_fin, x_fin, y_fin = pos_final // 12, (pos_final % 12) // 3, (pos_final % 12) % 3
        
        # Verificar se podemos remover do porto preenchido
        pode_mover = True
        if andar_ini < 1:
            if porto_preenchido[andar_ini + 1, x_ini, y_ini] == 1:
                pode_mover = False
        
        
        # Verificar se podemos adicionar à balsa
        if andar_fin > 0:
            if balsa[andar_fin - 1, x_fin, y_fin] == 0:
                pode_mover = False

        #Verfica se o lugar ta ocupado
        if balsa[andar_fin , x_fin, y_fin] == 1:
            pode_mover = False



                
        if pode_mover:
            # Atualizar porto_preenchido e balsa
            porto_preenchido[andar_ini, x_ini, y_ini] = 0
            balsa[andar_fin, x_fin, y_fin] = 1
            cal_movimentos += (
                              ((3-andar_ini)+(3-andar_fin)) #alturas
                              + (abs(x_ini - x_fin))
                              + ((2-y_ini)+(y_fin))
                              )
        else:
            cal_movimentos += 1000
            

    return cal_movimentos

def torneio(populacao, fitness, k=3):
    selecionados = random.sample(list(zip(populacao, fitness)), k)
    selecionados.sort(key=lambda x: x[1])
    return selecionados[0][0]

def pmx_crossover(parent1, parent2, start, end):
    size = end - start + 1
    child1 = parent1[:start] + [-1] * size + parent1[end+1:]
    child2 = parent2[:start] + [-1] * size + parent2[end+1:]
    
    mapping1 = parent1[start:end+1]
    mapping2 = parent2[start:end+1]
    
    # Copy the segment
    child1[start:end+1] = mapping2
    child2[start:end+1] = mapping1
    
    # Resolve conflicts
    for i in range(start):
        if child1[i] in mapping2:
            idx = mapping2.index(child1[i])
            while mapping1[idx] in mapping2:
                idx = mapping2.index(mapping1[idx])
            child1[i] = mapping1[idx]
    
    for i in range(end+1, len(parent1)):
        if child1[i] in mapping2:
            idx = mapping2.index(child1[i])
            while mapping1[idx] in mapping2:
                idx = mapping2.index(mapping1[idx])
            child1[i] = mapping1[idx]
    
    for i in range(start):
        if child2[i] in mapping1:
            idx = mapping1.index(child2[i])
            while mapping2[idx] in mapping1:
                idx = mapping1.index(mapping2[idx])
            child2[i] = mapping2[idx]
    
    for i in range(end+1, len(parent2)):
        if child2[i] in mapping1:
            idx = mapping1.index(child2[i])
            while mapping2[idx] in mapping1:
                idx = mapping1.index(mapping2[idx])
            child2[i] = mapping2[idx]
    
    return child1, child2

def cruzamento(pai, mae):
    filho1, filho2 = pai[:], mae[:]
    
    # Cruzamento para as posições iniciais (0-29)
    ponto_corte1 = random.randint(0, 29)
    ponto_corte2 = random.randint(0, 29)
    start, end = min(ponto_corte1, ponto_corte2), max(ponto_corte1, ponto_corte2)
    filho1[:30], filho2[:30] = pmx_crossover(pai[:30], mae[:30], start, end)
    
    # Cruzamento para as posições finais (30-59)
    ponto_corte1 = random.randint(30, 59)
    ponto_corte2 = random.randint(30, 59)
    start, end = min(ponto_corte1, ponto_corte2) - 30, max(ponto_corte1, ponto_corte2) - 30
    filho1[30:60], filho2[30:60] = pmx_crossover(pai[30:60], mae[30:60], start, end)
    
    # Cruzamento para a ordem de movimentação (60-89)
    ponto_corte1 = random.randint(60, 89)
    ponto_corte2 = random.randint(60, 89)
    start, end = min(ponto_corte1, ponto_corte2) - 60, max(ponto_corte1, ponto_corte2) - 60
    filho1[60:90], filho2[60:90] = pmx_crossover(pai[60:90], mae[60:90], start, end)
    
    return filho1, filho2

def mutacao(cromossomo, taxa_mutacao=0.01):
    if random.random() < taxa_mutacao:
        part = random.randint(0, 2)
        idx1, idx2 = random.sample(range(30), 2)
        if part == 0:
            cromossomo[idx1], cromossomo[idx2] = cromossomo[idx2], cromossomo[idx1]
        elif part == 1:
            cromossomo[30+idx1], cromossomo[30+idx2] = cromossomo[30+idx2], cromossomo[30+idx1]
        else:
            cromossomo[60+idx1], cromossomo[60+idx2] = cromossomo[60+idx2], cromossomo[60+idx1]
    return cromossomo

def algoritmo_genetico(tamanho_populacao, geracoes, taxa_mutacao=0.01):
    populacao = inicializar_populacao(tamanho_populacao)
    fitness = [preencher_porto(cromossomo_para_dataframe(cromossomo)) for cromossomo in populacao]
    
    for geracao in range(geracoes):
        nova_populacao = []
        while len(nova_populacao) < tamanho_populacao:
            pai = torneio(populacao, fitness)
            mae = torneio(populacao, fitness)
            
            filho1, filho2 = cruzamento(pai, mae)
            filho1 = mutacao(filho1, taxa_mutacao)
            filho2 = mutacao(filho2, taxa_mutacao)
            
            nova_populacao.append(filho1)
            nova_populacao.append(filho2)
        
        populacao = nova_populacao[:tamanho_populacao]
        fitness = [preencher_porto(cromossomo_para_dataframe(cromossomo)) for cromossomo in populacao]
        
        melhor_fit = min(fitness)
        print(f"Geração {geracao+1}: Melhor fitness = {melhor_fit}")
    
    melhor_cromossomo = populacao[fitness.index(min(fitness))]
    return melhor_cromossomo, min(fitness)

# Configurações
tamanho_populacao = 200
geracoes = 400
taxa_mutacao = 0.08

melhor_cromossomo, melhor_fit = algoritmo_genetico(tamanho_populacao, geracoes, taxa_mutacao)
print(f"Melhor cromossomo encontrado: {melhor_cromossomo}")
print(f"Melhor fitness: {melhor_fit}")


Geração 1: Melhor fitness = 13134
Geração 2: Melhor fitness = 13134
Geração 3: Melhor fitness = 13144
Geração 4: Melhor fitness = 11163
Geração 5: Melhor fitness = 12154
Geração 6: Melhor fitness = 14124
Geração 7: Melhor fitness = 12151
Geração 8: Melhor fitness = 12146
Geração 9: Melhor fitness = 12151
Geração 10: Melhor fitness = 11160
Geração 11: Melhor fitness = 13137
Geração 12: Melhor fitness = 11158
Geração 13: Melhor fitness = 11172
Geração 14: Melhor fitness = 11161
Geração 15: Melhor fitness = 11163
Geração 16: Melhor fitness = 13142
Geração 17: Melhor fitness = 12151
Geração 18: Melhor fitness = 11164
Geração 19: Melhor fitness = 11174
Geração 20: Melhor fitness = 11168
Geração 21: Melhor fitness = 9175
Geração 22: Melhor fitness = 9180
Geração 23: Melhor fitness = 12143
Geração 24: Melhor fitness = 11166
Geração 25: Melhor fitness = 8186
Geração 26: Melhor fitness = 9156
Geração 27: Melhor fitness = 8188
Geração 28: Melhor fitness = 9156
Geração 29: Melhor fitness = 9172
G

### Juntando antiga com a nova 

In [15]:
import random
import numpy as np
import pandas as pd

def gerar_cromossomo():
    cromossomo = []
    
    # Gera posições iniciais (0-29)
    posicoes_iniciais = list(range(30))
    random.shuffle(posicoes_iniciais)
    cromossomo.extend(posicoes_iniciais)
    
    # Gera posições finais (0-29)
    posicoes_finais = list(range(30))
    random.shuffle(posicoes_finais)
    cromossomo.extend(posicoes_finais)
    
    # Gera ordem de movimentação (0-29)
    ordem_movimentacao = list(range(30))
    random.shuffle(ordem_movimentacao)
    cromossomo.extend(ordem_movimentacao)
    
    return cromossomo

def cromossomo_para_dataframe(cromossomo):
    container = list(range(30))
    posicao_inicial = cromossomo[:30]
    posicao_final = cromossomo[30:60]
    ordem_movimentacao = cromossomo[60:90]
    
    df = pd.DataFrame({
        'Container': container,
        'Posição Inicial': posicao_inicial,
        'Posição Final': posicao_final,
        'Ordem de Movimentação': ordem_movimentacao
    })
    
    # Ordenar o DataFrame pela coluna 'Ordem de Movimentação'
    df = df.sort_values(by='Ordem de Movimentação')
    
    return df

def inicializar_populacao(tamanho_populacao):
    populacao = []
    for _ in range(tamanho_populacao):
        cromossomo = gerar_cromossomo()
        populacao.append(cromossomo)
    return populacao

def verifica_repetidos(cromossomo):
    intervalos = [
        (0, 29),
        (30, 59),
        (60, 89)
    ]
    
    for intervalo in intervalos:
        valores = cromossomo[intervalo[0]:intervalo[1]+1]
        if len(valores) != len(set(valores)):
            return True
    
    return False

def preencher_porto(df_cromossomo):
    # Inicializar o porto e balsa
    porto = np.zeros((2, 5, 3), dtype=int)
    balsa = np.zeros((3, 4, 3), dtype=int)
    porto_preenchido = np.ones((2, 5, 3), dtype=int)
    cal_movimentos = 0
    
    # Preencher o porto com os dados do cromossomo
    ordem_inicial = []
    ordem_final = []

    df_cromossomo = df_cromossomo.sort_values(by='Posição Inicial')
    for _, linha in df_cromossomo.iterrows():
        ordem_inicial.append(linha['Container'])
    idx = 0
    for andar in range(2):
        for x in range(5):
            for y in range(3):
                if idx < 30:
                    porto[andar, x, y] = ordem_inicial[idx]
                    idx += 1
    
    df_cromossomo = df_cromossomo.sort_values(by='Ordem de Movimentação')
    # Realizar movimentações
    for _, linha in df_cromossomo.iterrows():
        container = linha['Container']
        a_pos_inicial = linha['Posição Inicial']
        a_pos_final = linha['Posição Final']
        pos_inicial = linha['Posição Inicial'] - 1
        pos_final = linha['Posição Final'] - 1
        
        andar_ini, x_ini, y_ini = pos_inicial // 15, (pos_inicial % 15) // 3, (pos_inicial % 15) % 3
        andar_fin, x_fin, y_fin = pos_final // 12, (pos_final % 12) // 3, (pos_final % 12) % 3
        
        # Verificar se podemos remover do porto preenchido
        pode_mover = True
        if andar_ini < 1:
            if porto_preenchido[andar_ini + 1, x_ini, y_ini] == 1:
                pode_mover = False
        
        # Verificar se podemos adicionar à balsa
        if andar_fin > 0:
            if balsa[andar_fin - 1, x_fin, y_fin] == 0:
                pode_mover = False

        # Verifica se o lugar está ocupado
        if balsa[andar_fin , x_fin, y_fin] == 1:
            pode_mover = False

                
        if pode_mover:
            # Atualizar porto_preenchido e balsa
            porto_preenchido[andar_ini, x_ini, y_ini] = 0
            balsa[andar_fin, x_fin, y_fin] = 1
            cal_movimentos += (
                              ((3-andar_ini)+(3-andar_fin)) #alturas
                              + (abs(x_ini - x_fin))
                              + ((2-y_ini)+(y_fin))
                              )
        else:
            cal_movimentos += 1000
            

    return cal_movimentos

def torneio(populacao, fitness, k=3):
    selecionados = random.sample(list(zip(populacao, fitness)), k)
    selecionados.sort(key=lambda x: x[1])
    return selecionados[0][0]

def cruzamento_pmx(pai1, pai2, inicio, fim):
    tamanho = fim - inicio + 1
    filho1 = pai1[:inicio] + [-1] * tamanho + pai1[fim+1:]
    filho2 = pai2[:inicio] + [-1] * tamanho + pai2[fim+1:]
    
    mapeamento1 = pai1[inicio:fim+1]
    mapeamento2 = pai2[inicio:fim+1]
    
    # Copiar o segmento
    filho1[inicio:fim+1] = mapeamento2
    filho2[inicio:fim+1] = mapeamento1
    
    # Resolver conflitos para filho1
    for i in range(inicio):
        if filho1[i] in mapeamento2:
            idx = mapeamento2.index(filho1[i])
            while mapeamento1[idx] in mapeamento2:
                idx = mapeamento2.index(mapeamento1[idx])
            filho1[i] = mapeamento1[idx]
    
    for i in range(fim+1, len(pai1)):
        if filho1[i] in mapeamento2:
            idx = mapeamento2.index(filho1[i])
            while mapeamento1[idx] in mapeamento2:
                idx = mapeamento2.index(mapeamento1[idx])
            filho1[i] = mapeamento1[idx]
    
    # Resolver conflitos para filho2
    for i in range(inicio):
        if filho2[i] in mapeamento1:
            idx = mapeamento1.index(filho2[i])
            while mapeamento2[idx] in mapeamento1:
                idx = mapeamento1.index(mapeamento2[idx])
            filho2[i] = mapeamento2[idx]
    
    for i in range(fim+1, len(pai2)):
        if filho2[i] in mapeamento1:
            idx = mapeamento1.index(filho2[i])
            while mapeamento2[idx] in mapeamento1:
                idx = mapeamento1.index(mapeamento2[idx])
            filho2[i] = mapeamento2[idx]
    
    return filho1, filho2

def cruzamento(pai, mae):
    filho1, filho2 = pai[:], mae[:]
    
    # Cruzamento para as posições iniciais (0-29)
    ponto_corte1 = random.randint(0, 29)
    ponto_corte2 = random.randint(0, 29)
    inicio, fim = min(ponto_corte1, ponto_corte2), max(ponto_corte1, ponto_corte2)
    filho1[:30], filho2[:30] = cruzamento_pmx(pai[:30], mae[:30], inicio, fim)
    
    # Cruzamento para as posições finais (30-59)
    ponto_corte1 = random.randint(30, 59)
    ponto_corte2 = random.randint(30, 59)
    inicio, fim = min(ponto_corte1, ponto_corte2) - 30, max(ponto_corte1, ponto_corte2) - 30
    filho1[30:60], filho2[30:60] = cruzamento_pmx(pai[30:60], mae[30:60], inicio, fim)
    
    # Cruzamento para a ordem de movimentação (60-89)
    ponto_corte1 = random.randint(60, 89)
    ponto_corte2 = random.randint(60, 89)
    inicio, fim = min(ponto_corte1, ponto_corte2) - 60, max(ponto_corte1, ponto_corte2) - 60
    filho1[60:90], filho2[60:90] = cruzamento_pmx(pai[60:90], mae[60:90], inicio, fim)
    
    return filho1, filho2

def mutacao(cromossomo, taxa_mutacao=0.01):
    if random.random() < taxa_mutacao:
        part = random.randint(0, 2)
        idx1, idx2 = random.sample(range(30), 2)
        if part == 0:
            cromossomo[idx1], cromossomo[idx2] = cromossomo[idx2], cromossomo[idx1]
        elif part == 1:
            cromossomo[30+idx1], cromossomo[30+idx2] = cromossomo[30+idx2], cromossomo[30+idx1]
        else:
            cromossomo[60+idx1], cromossomo[60+idx2] = cromossomo[60+idx2], cromossomo[60+idx1]
    return cromossomo

def algoritmo_genetico(tamanho_populacao, geracoes, taxa_mutacao):
    populacao = inicializar_populacao(tamanho_populacao)
    fitness = [preencher_porto(cromossomo_para_dataframe(cromossomo)) for cromossomo in populacao]
    
    for geracao in range(geracoes):
        nova_populacao = []
        while len(nova_populacao) < tamanho_populacao:
            pai = torneio(populacao, fitness)
            mae = torneio(populacao, fitness)
            
            filho1, filho2 = cruzamento(pai, mae)
            filho1 = mutacao(filho1, taxa_mutacao)
            filho2 = mutacao(filho2, taxa_mutacao)
            
            nova_populacao.append(filho1)
            nova_populacao.append(filho2)
        
        # Combinar a metade melhor da geração antiga com a metade melhor da nova geração
        populacao.extend(nova_populacao)
        fitness = [preencher_porto(cromossomo_para_dataframe(cromossomo)) for cromossomo in populacao]
        
        # Ordenar pela fitness e selecionar os melhores
        populacao_ordenada = sorted(zip(populacao, fitness), key=lambda x: x[1])
        populacao = [cromossomo for cromossomo, _ in populacao_ordenada[:tamanho_populacao]]
        fitness = [fit for _, fit in populacao_ordenada[:tamanho_populacao]]
        
        melhor_fit = min(fitness)
        print(f"Geração {geracao+1}: Melhor fitness = {melhor_fit}")
    
    melhor_cromossomo = populacao[fitness.index(min(fitness))]
    return melhor_cromossomo, min(fitness)

# Configurações
tamanho_populacao = 100
geracoes = 200
taxa_mutacao = 0.1

melhor_cromossomo, melhor_fit = algoritmo_genetico(tamanho_populacao, geracoes, taxa_mutacao)
print(f"Melhor cromossomo encontrado: {melhor_cromossomo}")
print(f"Melhor fitness: {melhor_fit}")


Geração 1: Melhor fitness = 12151
Geração 2: Melhor fitness = 12151
Geração 3: Melhor fitness = 12151
Geração 4: Melhor fitness = 12151
Geração 5: Melhor fitness = 12151
Geração 6: Melhor fitness = 9184
Geração 7: Melhor fitness = 9184
Geração 8: Melhor fitness = 9184
Geração 9: Melhor fitness = 9184
Geração 10: Melhor fitness = 9184
Geração 11: Melhor fitness = 9184
Geração 12: Melhor fitness = 9184
Geração 13: Melhor fitness = 9184
Geração 14: Melhor fitness = 9184
Geração 15: Melhor fitness = 9184
Geração 16: Melhor fitness = 9184
Geração 17: Melhor fitness = 9171
Geração 18: Melhor fitness = 9169
Geração 19: Melhor fitness = 8183
Geração 20: Melhor fitness = 7194
Geração 21: Melhor fitness = 7189
Geração 22: Melhor fitness = 7189
Geração 23: Melhor fitness = 6203
Geração 24: Melhor fitness = 6203
Geração 25: Melhor fitness = 6196
Geração 26: Melhor fitness = 5203
Geração 27: Melhor fitness = 5203
Geração 28: Melhor fitness = 5203
Geração 29: Melhor fitness = 4214
Geração 30: Melhor

#### Verifica se ha numeros repetidos


In [16]:
def verifica_repetidos(cromossomo):
    intervalos = [
        (0, 29),
        (30, 59),
        (60, 89)
    ]
    
    for intervalo in intervalos:
        valores = cromossomo[intervalo[0]:intervalo[1]+1]
        if len(valores) != len(set(valores)):
            return True
    
    return False

if verifica_repetidos(melhor_cromossomo):
    print("Valores repetidos encontrados!")
else:
    print("Não há valores repetidos.")

df_cromossomo = cromossomo_para_dataframe(melhor_cromossomo)
print("DataFrame do Cromossomo:")
print(df_cromossomo)


Não há valores repetidos.
DataFrame do Cromossomo:
    Container  Posição Inicial  Posição Final  Ordem de Movimentação
12         12               18              2                      0
8           8               15             18                      1
26         26               27             11                      2
22         22               23              9                      3
15         15               26             12                      4
10         10               19              4                      5
20         20                8              8                      6
4           4               29             10                      7
23         23               25             16                      8
2           2               24             20                      9
9           9                0             15                     10
6           6               17              1                     11
13         13               22              7       

### Simular Melhor cromomossomo

In [18]:
def Simular_preencher_porto(df_cromossomo):
    # Inicializar o porto e balsa
    porto = np.zeros((2, 5, 3), dtype=int)
    balsa = np.zeros((3, 4, 3), dtype=int)
    porto_preenchido = np.ones((2, 5, 3), dtype=int)
    
    
    # Preencher o porto com os dados do cromossomo
    ordem_inicial = []
    ordem_final = []
    

    df_cromossomo = df_cromossomo.sort_values(by='Posição Inicial')
    for _, linha in df_cromossomo.iterrows():
        ordem_inicial.append(linha['Container'])
    idx = 0
    for andar in range(2):
        for x in range(5):
            for y in range(3):
                if idx < 30:
                    porto[andar, x, y] = ordem_inicial[idx]
                    idx += 1
    
    df_cromossomo = df_cromossomo.sort_values(by='Ordem de Movimentação')
    # Realizar movimentações
    for _, linha in df_cromossomo.iterrows():
        container = linha['Container']
        a_pos_inicial = linha['Posição Inicial']
        a_pos_final = linha['Posição Final']
        pos_inicial = linha['Posição Inicial'] 
        pos_final = linha['Posição Final'] 
        
        andar_ini, x_ini, y_ini = pos_inicial // 15, (pos_inicial % 15) // 3, (pos_inicial % 15) % 3
        andar_fin, x_fin, y_fin = pos_final // 12, (pos_final % 12) // 3, (pos_final % 12) % 3
        
        # Verificar se podemos remover do porto preenchido
        pode_mover = True
        if andar_ini < 1:
            if porto_preenchido[andar_ini + 1, x_ini, y_ini] == 1:
                pode_mover = False
        
        # Verificar se podemos adicionar à balsa
        if andar_fin > 0:
            if balsa[andar_fin - 1, x_fin, y_fin] == 0:
                pode_mover = False
        
        if pode_mover:
            # Atualizar porto_preenchido e balsa
            porto_preenchido[andar_ini, x_ini, y_ini] = 0
            balsa[andar_fin, x_fin, y_fin] = 1
            
            print(f"Movendo container {container} de ({a_pos_inicial})({pos_inicial})({andar_ini}, {x_ini}, {y_ini}) para ({a_pos_final})({pos_final})({andar_fin}, {x_fin}, {y_fin})")
            print("Estado atual do Porto Preenchido:")
            print(porto_preenchido)
            print("Estado atual da Balsa:")
            print(balsa)
            print("\n")
        else:
            print(f"não moveu container {container} de ({a_pos_inicial})({pos_inicial})({andar_ini}, {x_ini}, {y_ini}) para ({a_pos_final})({pos_final})({andar_fin}, {x_fin}, {y_fin})")


Simular_preencher_porto(cromossomo_para_dataframe(melhor_cromossomo))

Movendo container 12 de (18)(18)(1, 1, 0) para (2)(2)(0, 0, 2)
Estado atual do Porto Preenchido:
[[[1 1 1]
  [1 1 1]
  [1 1 1]
  [1 1 1]
  [1 1 1]]

 [[1 1 1]
  [0 1 1]
  [1 1 1]
  [1 1 1]
  [1 1 1]]]
Estado atual da Balsa:
[[[0 0 1]
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]]


não moveu container 8 de (15)(15)(1, 0, 0) para (18)(18)(1, 2, 0)
Movendo container 26 de (27)(27)(1, 4, 0) para (11)(11)(0, 3, 2)
Estado atual do Porto Preenchido:
[[[1 1 1]
  [1 1 1]
  [1 1 1]
  [1 1 1]
  [1 1 1]]

 [[1 1 1]
  [0 1 1]
  [1 1 1]
  [1 1 1]
  [0 1 1]]]
Estado atual da Balsa:
[[[0 0 1]
  [0 0 0]
  [0 0 0]
  [0 0 1]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]]


Movendo container 22 de (23)(23)(1, 2, 2) para (9)(9)(0, 3, 0)
Estado atual do Porto Preenchido:
[[[1 1 1]
  [1 1 1]
  [1 1 1]
  [1 1 1]
  [1 1 1]]

 [[1 1 1]
  [0 1 1]
  [1 1 0]
  [1 1 1]
  [0 1 1]]]
Estado atual da Balsa:
[[[0

In [ ]:
pos = 27
andar_ini, x_ini, y_ini = pos // 12, (pos % 12) // 3, (pos % 12) % 3
print(andar_ini," ",x_ini," ",y_ini)

2   1   0
